In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import EfficientNetB7, EfficientNetB1, EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [ ]:
#資料前處理
#zoom_range 聚焦 rotation_range 旋轉 horizontal_flip 水平翻轉 vertical_flip 垂直翻轉

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.,
    rotation_range=0.,
    width_shift_range=0., 
    height_shift_range=0.)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
#target_size 和 batch_size 可調整
train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(640, 480),
        batch_size=16,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'dataset/val',
        target_size=(640, 480),
        batch_size=16,
        class_mode='categorical')

In [ ]:
epochs = 200

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=1, mode='auto')
filepath = "weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
csvlog = tf.keras.callbacks.CSVLogger('PI_AOI_train_log.csv')
callbacks = [csvlog, checkpoint]

In [ ]:
#讀取網路架構，可選擇其他的網路架構，請參考 keras API:https://www.tensorflow.org/api_docs/python/tf/keras/applications
#選擇其他網路架構時，ImageDataGenerator的preprocessing_function需要改成對應網路的preprocess_input
#第一次訓練時執行此行
model = EfficientNetB0(include_top = False, weights='imagenet', pooling = 'avg', classes = 2)

x = model.output
#Dropout層可加可不加，如果overfitting就加
x = layers.Dropout(0.2, name='top_dropout')(x)
output = Dense(units = 2, activation="softmax")(x)
model = Model(inputs=model.input, outputs = output, name='efficientnet')
model.summary()

In [ ]:
#如果要從以前訓練的模型開始訓練，執行此行，檔名.hdf5
#model = tf.keras.models.load_model("weights-improvement-20-0.90.hdf5")

In [ ]:
#learning_rate 可調整
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
#開始訓練
#model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    verbose=1,
    use_multiprocessing=True,
    workers=4,
    callbacks=callbacks
)
